# Model Deployment with Flask

This notebook demonstrates how to deploy a trained machine learning model as a web API using **Flask** — a lightweight Python web framework.

### Objectives
- Build a simple Flask API endpoint.
- Load a trained model and return predictions.
- Test the Flask app locally.

In [1]:
# !pip install flask scikit-learn joblib

## Step 1: Train and Save a Model

In [2]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import joblib

# Load data
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, 'iris_model.pkl')

## Step 2: Create Flask App

In [3]:
from flask import Flask, request, jsonify
import numpy as np

app = Flask(__name__)
model = joblib.load('iris_model.pkl')

@app.route('/')
def home():
    return "<h2>Flask Model Deployment</h2><p>Send POST to /predict</p>"

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    features = np.array(data['features']).reshape(1, -1)
    prediction = model.predict(features)[0]
    return jsonify({'prediction': int(prediction), 'class': iris.target_names[prediction]})

if __name__ == '__main__':
    app.run(debug=True)

## Step 3: Test API Locally

In [4]:
# import requests
# data = {'features': [5.1, 3.5, 1.4, 0.2]}
# response = requests.post('http://127.0.0.1:5000/predict', json=data)
# print(response.json())

### Notes
- Use Gunicorn + Nginx for production.
- Can be containerized with Docker:
```dockerfile
FROM python:3.9
WORKDIR /app
COPY . .
RUN pip install flask scikit-learn joblib
CMD ["python", "app.py"]
```